<a href="https://colab.research.google.com/github/Tsanyu20/bachelor_thesis/blob/main/Copy_of_Preview_Copy_of_Social_Media_Analysis_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Social Media Analysis 101

**Author**: [Yara Kyrychenko](https://www.sdmlab.psychol.cam.ac.uk/staff/yara-kyrychenko).



**Contents**:
1.   Scraping Telegram
2.   Text Analysis
3.   Statistics

Add your secrets in the panel to the left under the key sign.
  - You will need: `api_id`, `api_hash`, and `openai_key`.
  
Change runtime to GPU if you want to use GPU for open source zero shot classification.

**Don't forget to validate your LLM labels for real studies!!!**


In [ ]:
!git clone https://github.com/Tsanyu20/bachelor_thesis.git

In [ ]:
!pip install -r /content/bachelor_thesis/requirements.txt

In [ ]:
import json
import re

years = [2022, 2023, 2024, 2025]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
dates = {}

# for year in years:
#   for month in months:
#     dates[f"{year}-{month}"] = []
with open('/content/bachelor_thesis/deoccupied/VKhersone.json', encoding='utf-8') as f:
  data = json.load(f)

for dic in data:
  for year in years:
    for month in months:
      if re.findall(f"{year}-{month}", dic['date']):
        content = []
        if dic['text']:
          content.append(dic['text'])
        if dic['caption']:
          content.append(dic['caption'])
        if content:
          if dates.get(f"{year}-{month}"):
            dates[f"{year}-{month}"].append(content)
          else :
            dates[f"{year}-{month}"] = [content]


In [ ]:
dates

In [ ]:
import json
import re

def extract_date(json_files):
  years = [2022, 2023, 2024, 2025]
  months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  dates = {}

  for file in json_files:
    try:
      with open(file,  encoding='utf-8') as f:
        data = json.load(f)

      for dic in data:
        for year in years:
          for month in months:
            if re.findall(f"{year}-{month}", dic['date']):
              content = []
              if dic['text']:
                content.append(dic['text'])
              if dic['caption']:
                content.append(dic['caption'])
              if content:
                if dates.get(f"{year}-{month}"):
                  dates[f"{year}-{month}"].append(content)
                else :
                  dates[f"{year}-{month}"] = [content]

    except json.JSONDecodeError as e:
      print(f"Error decoding JSON in file {file}: {e}")

  return dates

In [ ]:
import glob
occupied_sample = glob.glob("/content/bachelor_thesis/occupied/*.json")
oc_date = extract_date(occupied_sample)
deoccupied_sample = glob.glob("/content/bachelor_thesis/deoccupied/*.json")
de_date = extract_date(deoccupied_sample)

In [ ]:
print('Occupied:')
for key, value in oc_date.items():
  print(f"{key}: {len(value)}")
print('------------------------------')
print('Deoccupied:')
for key, value in de_date.items():
  print(f"{key}: {len(value)}")

## Extract content from json files and identify language used

In [ ]:
import glob
import json

# Use this function to get all non-empty text/caption
def extract_text(json_files):
  all_content=[]
  for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for dic in data:
                  if dic['text']:
                    all_content.append(dic['text'])
                  if dic['caption']:
                    all_content.append(dic['caption'])

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {file}: {e}")

  if all_content:
    return all_content
  else:
      print("No data found in JSON files.")
      return None

# oc_json = glob.glob('/content/bachelor_thesis/occupied/*.json')
# oc_content = extract_text(oc_json)
de_json = glob.glob('/content/bachelor_thesis/deoccupied/*.json')
de_content = extract_text(de_json)

In [ ]:
from langdetect import detect

count_ru = 0
count_uk = 0

de_uk = []
for sentence in de_content:
  try :
    # Detect if it's uk or ru
    lang = detect(sentence)
    if (lang == 'ru'):
      count_ru += 1
    elif (lang == 'uk'):
      count_uk += 1
      de_uk.append(sentence)
  except Exception as e:
    # Remove the messages that can't be distinguished to one language
    de_content.remove(sentence)

print(f"Count for Russian: {count_ru}, Count for Ukrainian {count_uk}")

## Tfidf Vectorization


### In Ukrainian

In [ ]:
!python -m spacy download uk_core_news_sm

In [ ]:
# @title Perform lemmatization on dataset
import spacy
from gensim import corpora
from gensim import models
import gensim

#nlp_r = spacy.load('ru_core_news_sm')
nlp_u = spacy.load('uk_core_news_sm')

doc = []
for sen in de_uk:
  doc.append(nlp_u(sen))

lemmatizer = nlp_u.get_pipe("lemmatizer")
processed_corpus_u =[[token.lemma_ for token in sen] for sen in doc]

# Make the processed list as dictionary
dictionary_u = corpora.Dictionary(processed_corpus_u)
bow_corpus_u = [dictionary_u.doc2bow(text) for text in processed_corpus_u]


# train the model
tfidf = models.TfidfModel(bow_corpus_u)
corpus_tfidf = tfidf[bow_corpus_u]

# transform into numpy array
"""corpus_csr = gensim.matutils.corpus2csc(corpus_tfidf)
corpus_numpy = corpus_csr.T.toarray()"""
tfidf_matrix_u = gensim.matutils.corpus2dense(corpus_tfidf, num_terms=len(dictionary_u)).T

### In English

In [ ]:
# @title Translate into EN-US to perform vectorization
import deepl

auth_key = "a7c78cad-9348-425d-be66-88410ea035ac:fx"  # Replace with your key
deepl_client = deepl.DeepLClient(auth_key)

doc_e = []
for sen in de_uk:
  doc_e.append(deepl_client.translate_text(sen, target_lang="EN-US").text)

doc_e[0]

In [ ]:
# @title Use TfidfVectorizer from sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

vectorizer = TfidfVectorizer(max_df=0.5, min_df=5, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(doc_e)

### Use spacy and gensim to preprocess and tokenize

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from gensim import corpora
from gensim import models
import gensim

nlp = spacy.load('en_core_web_sm')

documents=[]
for sen in doc_e:
  documents.append(nlp(sen))

# Lemmatize
lemmatizer = nlp.get_pipe("lemmatizer")
processed_corpus = [[token.lemma_ for token in sen] for sen in documents]

# Define dictionary and count the frequency of each word
dictionary = corpora.Dictionary(processed_corpus)
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

# train the model
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

# Transform into numpy array
tfidf_matrix_g = gensim.matutils.corpus2dense(corpus_tfidf, num_terms=len(dictionary)).T

## Compare the plots

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)

pca_result = pca.fit_transform(tfidf_matrix) # with sklearn
pca_result_g = pca.fit_transform(tfidf_matrix_g) # with gensim
pca_result_u = pca.fit_transform(tfidf_matrix_u) # in ukranian

In [ ]:
# Use sklearn TfidfVectorizer
plt.figure(figsize=(10, 7))
plt.scatter(pca_result[:, 0], pca_result[:, 1], c='blue', edgecolor='k', s=50)
for i, txt in enumerate(doc_e):
    plt.annotate(i, (pca_result[i, 0], pca_result[i, 1]))


# Use spacy and gensim
plt.scatter(pca_result_g[:, 0], pca_result_g[:, 1], c='orange', edgecolor='k', s=50)
for i, txt in enumerate(doc_e):
    plt.annotate(i, (pca_result_g[i, 0], pca_result_g[i, 1]))

# Original Language
plt.scatter(pca_result_u[:, 0], pca_result_u[:, 1], c='green', edgecolor='k', s=50)
for i, txt in enumerate(doc_e):
    plt.annotate(i, (pca_result_u[i, 0], pca_result_u[i, 1]))

labels = ['sklearn', 'gensim', 'ukranian']
plt.legend(labels)
plt.title('PCA of TF-IDF Matrix')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.show()

In [ ]:
# KMeans clustering
from sklearn.cluster import KMeans

kmeans = KMeans(
    n_clusters=4,
    max_iter=100,
    n_init=5,
).fit(pca_result)

labels = kmeans.predict(pca_result)
# Make a scatter plot of xs and ys, using labels to define the colors
plt.scatter(pca_result[:, 0], pca_result[:,1],alpha=0.5)
# Assign the cluster centers: centroids
centroids = kmeans.cluster_centers_
# Assign the columns of centroids: centroids_x, centroids_y
centroids_x = centroids[:,0]
centroids_y = centroids[:,1]
# Make a scatter plot of centroids_x and centroids_y
plt.scatter(centroids_x,centroids_y,marker='D',s=50)
plt.show()

## Google Fact Check Tools API

In [ ]:
import requests
#https://developers.google.com/fact-check/tools/api/reference/rest/v1alpha1/claims/search
from google.colab import userdata

def fact_check_search(query, api_key):
  # Base URL for the API
  url = "https://factchecktools.googleapis.com/v1alpha1/claims:search"
  # Parameters for the API request
  params = {
    'query': query, # Search term (string)
    'languageCode': 'en', # Language (optional)
    'key': api_key, # Your API key
    'pageSize': 10 # Number of results returned,
    # 'maxAgeDays': 30 # Maximum age of the results in days
    }

  # Make the GET request
  response = requests.get(url, params=params)
  # Handle response
  if response.status_code == 200:
    return response.json()
  else:
    print(f"Error: {response.status_code}, {response.text}")
    return None

# Replace with your API key
API_KEY = userdata.get('fact_check_apikey')
search_query = "Ukraine"
result = fact_check_search(search_query, API_KEY)

# Print results
if result:
  for claim in result.get('claims', []):
    print(f"Claim: {claim.get('text')}")
    print(f"Claimant: {claim.get('claimant')}")
    print(f"Rating: {claim.get('claimReview', [{}])[0].get('textualRating')}")
    print(f"URL: {claim.get('claimReview', [{}])[0].get('url')}")
    print("-" * 40)


In [ ]:
# @title Alternative: Use an open-source Library to get access to Fact Check Tools
!pip install git+https://github.com/GONZOsint/factcheckexplorer.git

In [ ]:
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 1. Scraping Telegram

In [ ]:
! pip install pyrogram
! pip3 install -U tgcrypto

Discover Telegram Channels: https://tgstat.ru/en

Check out `pyrogram`: https://docs.pyrogram.org/

Get the Telegram API id and hash: https://core.telegram.org/api/obtaining_api_id

In [ ]:
from pyrogram import Client
from google.colab import userdata

api_id = int(userdata.get('api_id'))
api_hash = userdata.get('api_hash')

app = Client("UkraineRussiaCrisis", api_id=api_id, api_hash=api_hash)

In [ ]:
# Use the existing event loop in Jupyter/IPython
import nest_asyncio
import asyncio
import json
from pyrogram.types import Message

def get_reactions(message):
    reactions = {}
    if hasattr(message, "reactions") and message.reactions is not None:
        for reaction in message.reactions.reactions:
            reactions[reaction.emoji] = reaction.count
    return reactions

def get_urls(entities):
    urls = []
    if entities:
        for entity in entities:
            try:
                if entity.url is not None:
                    urls.append(entity.url)
            except:
                pass
    return urls

async def get_channel_history(chat_id, output_file=None, message_limit=15):
    output_file = chat_id if output_file is None else output_file

    if not app.is_connected:
        await app.start()

    chat = await app.get_chat(chat_id)
    followers_count = chat.members_count or 0

    all_messages, message_dicts = [], []
    async for message in app.get_chat_history(chat_id, limit=message_limit):

        all_messages.append(message)

        message_dict = {
            "id": message.id,
            "text": message.text,
            "date": message.date.isoformat(),
            "caption": message.caption,
            "views": message.views,
            "forwards": message.forwards,
            "reactions": get_reactions(message),
            "channel_memebers_count": followers_count,
            "channel_username": message.chat.username,
            "sender_username": message.sender_chat.username,
            "channel_id": message.chat.id,
            "sender_id": message.sender_chat.id,
            "channel_is_verified": message.chat.is_verified,
            "sender_is_verified": message.sender_chat.is_verified,
            "reply_to_message_id": message.reply_to_message_id,
            "reply_to_top_message_id": message.reply_to_top_message_id,
            "video": message.video.file_id if message.video else None,
            "photo": message.photo.file_id if message.photo else None,
            "sticker": message.sticker.file_id if message.sticker else None,
            "url": get_urls(message.caption_entities)

        }

        message_dicts.append(message_dict)

    with open(f'{output_file}.json', 'w', encoding='utf-8') as f:
        json.dump(message_dicts, f, ensure_ascii=False, indent=4)

    if app.is_connected:
        await app.stop()



In [ ]:
channels = pd.read_csv('/content/bachelor_thesis/UkraineTGChannels.csv')
occupied = channels[channels['Category']=='Occupied'].Username.to_list()
deoccupied = channels[channels['Category']=='De-occupied'].Username.to_list()

In [ ]:
np.random.seed(42)

occupied_sample = np.random.choice(occupied, min(25, len(occupied)), replace=False)
deoccupied_sample = np.random.choice(deoccupied, min(25, len(deoccupied)),replace=False)

print(f"Sampled {len(occupied_sample)} occupied channels: {occupied_sample}")
print(f"Sampled {len(deoccupied_sample)} deoccupied channels: {deoccupied_sample}")

In [ ]:
nest_asyncio.apply()
await get_channel_history("rian_ru",message_limit=100)

In [ ]:
nest_asyncio.apply()
for chat_id in occupied_sample:
    try:
      await get_channel_history(chat_id,output_file=f'/content/bachelor_thesis/occupied/{chat_id}',message_limit=100)
    except Exception as e:
      print(f"Error processing channel {chat_id}: {e}")

for chat_id in deoccupied_sample:
    try:
      await get_channel_history(chat_id,output_file=f'/content/bachelor_thesis/deoccupied/{chat_id}',message_limit=100)
    except Exception as e:
      print(f"Error processing channel {chat_id}: {e}")

In [5]:
import glob
import json
import pandas as pd

def json_to_df(json_files):
    all_data = []
    for file in json_files:
        try:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_data.extend(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {file}: {e}")

    if all_data:
      return pd.DataFrame(all_data)
    else:
      print("No data found in JSON files.")
      return None



In [31]:
json_files = glob.glob("./occupied/*.json")
oc_df = json_to_df(json_files)
json_files = glob.glob("./deoccupied/*.json")
deoc_df = json_to_df(json_files)

In [32]:
oc_df['category'] = 'occupied'
deoc_df['category'] = 'deoccupied'
df = pd.concat([oc_df, deoc_df], ignore_index=True)

In [36]:
small_df = pd.read_csv('results_df_all_custom.csv')
small_df

,Unnamed: 0.1,Unnamed: 0,id,text,date,caption,views,forwards,reactions,channel_memebers_count,...,polarization,impersonation,conspiracy,trolling,pro-ukrainian,pro-russian,pro-ukrainian-mDeBERTa,pro-russian-mDeBERTa,pro-ukrainian-custom,pro-russian-custom
0,0,0,302,NaN,2024-12-15T08:01:07,💡❗Графіки відключень світла по Чернігівщині на...,832.0,1.0,{},444,...,0,0,0,0,0,0,0.511689,0.488311,0.995916,0.004084
1,1,1,7336,💔 С 1 января госпошлина на развод вырастет в 8...,2024-12-31T07:58:39,NaN,226.0,1.0,{'🤬': 1},1729,...,0,0,0,0,0,0,0.455062,0.544938,0.007027,0.992972
2,2,2,25763,❤️ Взятие Бердина - вброс гребней и паникеров....,2025-01-05T09:17:31,NaN,850.0,1.0,"{'🙏': 15, '🥴': 1}",2799,...,3,0,1,2,0,4,0.638906,0.361094,0.006853,0.993147
3,3,3,7096,NaN,2025-01-01T11:30:23,Фермери Херсонщини планують розширити площі пі...,423.0,0.0,{'👍': 3},4752,...,0,0,0,0,0,0,0.477264,0.522736,0.995770,0.004230
4,4,4,1967,Семенівка - КАБи в місто... \n\nДеталі з офіці...,2025-01-04T19:22:59,NaN,6334.0,23.0,"{'😱': 36, '😭': 31, '🤬': 26, '💯': 3, '🤣': 1}",6118,...,3,0,0,2,4,0,0.643907,0.356093,0.995826,0.004174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,495,21901,NaN,2025-01-08T18:28:18,Новогодние праздники подошли к концу — атмосфе...,60207.0,232.0,"{'🙏': 86, '👏': 71, '❤': 67, '🔥': 62, '👍': 38, ...",308903,...,0,0,0,0,0,0,0.588331,0.411669,0.007292,0.992708
496,496,496,21855,NaN,2025-01-06T07:05:58,⚡️Освобождение Курахова лишило ВСУ возможности...,59669.0,214.0,"{'🙏': 107, '\U0001fae1': 80, '🔥': 67, '👏': 63,...",308903,...,3,0,1,2,0,5,0.477560,0.522440,0.006721,0.993279
497,497,497,6538,NaN,2024-12-29T07:55:42,"Дорогие земляки, поздравления с наступающим Но...",36084.0,97.0,"{'👍': 232, '❤': 89, '🥰': 14, '😢': 13, '👏': 8, ...",15812,...,0,0,0,0,0,3,0.423041,0.576959,0.006717,0.993283
498,498,498,1702,❗️Сообщают о взрыве в Ровном,2022-10-10T08:16:49,NaN,1442.0,2.0,{'😢': 1},737,...,0,0,0,0,0,0,0.553514,0.446486,0.223716,0.776284


## 2. Text Analysis




In [ ]:
df[['text', 'caption']] = df[['text', 'caption']].fillna("")
df['all_text'] = df['caption'] + ' ' + df['text']
df = df[df['all_text'] != ' ']
df['all_text_prep'] = df['all_text'].apply(lambda x: x[:512] + '...' if len(x) > 512 else x)

In [ ]:
n_samples = 250
dfsmall = df.groupby('category').apply(lambda x: x.sample(n=n_samples, random_state=42)).reset_index(drop=True)

### 2.1 ChatGPT

Get openai api key: https://platform.openai.com/docs/overview

In [ ]:
%%capture
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import openai
from openai import OpenAI

openai.api_key = userdata.get('openai_key')
client = OpenAI(api_key=userdata.get('openai_key'))

In [ ]:
def analyze_texts(texts):
    results = []
    for text in texts:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are an assistant that analyzes Telegram posts for specific linguistic categories. "
                        "For each input text, return a JSON object with the following keys: "
                        "'discrediting', 'emotion', 'polarization', 'impersonation', 'conspiracy', 'trolling', "
                        "'pro-ukrainian', 'pro-russian'."
                        "Each key should have a value between 0 and 6 indicating the intensity of the category."
                        "Discrediting refers to discrediting opponents or creating doubt about them."
                        "Emotion refers to the use of outrage or highly emotive language to manipulate people."
                        "Polarization here means using divisive issues to drive a wedge between two groups."
                        "Impersonation refers to misusing the identity of politicians, experts, or celebrities."
                        "Conspiracy refers to casting doubt on mainstream narratives by providing an attractive story in which a small sinister group of people is responsible for doing harm to many."
                        "Trolling is aimed at eliciting reactions from people by provoking them."
                        "Finally, rate if the post has a pro-Ukrainian or a pro-Russian view on the war in Ukraine."
                    ),
                },
                {"role": "user", "content": f"Analyze this post: {text}"},
            ],
            temperature=0.0,
            response_format = { "type": "json_object" },
        )
        results.append(response.choices[0].message.content)
    return results

In [ ]:
# 500 posts took around 16 minutes
results = analyze_texts(dfsmall.all_text_prep)
results_df = pd.DataFrame([json.loads(result) for result in results])

In [ ]:
results_df_all = pd.concat([dfsmall, results_df], axis=1)

In [ ]:
for i, row in results_df_all[:10].iterrows():
    print(f"Category: {row['category']}")
    print(f"Post: {row['all_text']}")
    print(f"Discrediting: {row['discrediting']}")
    print(f"Emotion: {row['emotion']}")
    print(f"Polarization: {row['polarization']}")
    print(f"Impersonation: {row['impersonation']}")
    print(f"Conspiracy: {row['conspiracy']}")
    print(f"Trolling: {row['trolling']}")
    print(f"Pro-Ukrainian: {row['pro-ukrainian']}")
    print(f"Pro-Russian: {row['pro-russian']}")
    print()


### 2.2 Open source models

In [ ]:
results_df_all = pd.read_csv('results_df_all_custom.csv')

In [ ]:
! pip install 'transformers[torch]'

In [ ]:
from transformers import pipeline
import torch

classifier = pipeline("zero-shot-classification",
                      model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
                      device=0 if torch.cuda.is_available() else -1)
sequence_to_classify = "Русня - не люди."
candidate_labels = ["pro-russian",'pro-ukrainian']
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)


In [ ]:
sequence_to_classify = "Русня - не люди."
candidate_labels = ["solidarity with Ukraine or Ukrainians",'other']
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

In [ ]:
sequence_to_classify = "Русня - не люди."
candidate_labels = ["hostility towards Russia or Russians",'other']
output = classifier(sequence_to_classify, candidate_labels, multi_label=False)
print(output)

In [ ]:
candidate_labels = ["pro-russian",'pro-ukrainian']
output = classifier(list(results_df_all['all_text_prep'].values), candidate_labels, multi_label=False)
print(output)

In [ ]:
results2 = pd.DataFrame([
    {
        **{label: score for label, score in zip(item['labels'], item['scores'])}
    }
    for item in output
])

In [ ]:
results2

In [ ]:
results2.columns = ['pro-ukrainian-mDeBERTa','pro-russian-mDeBERTa']

### 2.3 Custom models

Fine-tuning a language model: https://huggingface.co/docs/transformers/en/training

Ukraine POV model: https://huggingface.co/YaraKyrychenko/ukraine-war-pov

In [ ]:
custom_classifier = pipeline("text-classification",
                             model='YaraKyrychenko/ukraine-war-pov',
                             device=0 if torch.cuda.is_available() else -1)

In [ ]:
custom_output = custom_classifier(list(results_df_all['all_text_prep'].values),return_all_scores=True)
print(custom_output)

In [ ]:
results_custom = pd.DataFrame([
    {item['label']: item['score'] for item in row}
    for row in custom_output
])

In [ ]:
results_custom

In [ ]:
results_custom.columns = ['pro-ukrainian-custom','pro-russian-custom']

In [ ]:
results_df_all = pd.concat([results_df_all, results2,results_custom], axis=1)

### 2.4 Comparison

In [ ]:
results_df_all['ur'] = results_df_all['pro-ukrainian'] - results_df_all['pro-russian']
results_df_all['ua_bin'] = results_df_all['ur'].apply(lambda x: 1 if x > 3 else 0)
results_df_all['ru_bin'] = results_df_all['ur'].apply(lambda x: 1 if x < -3 else 0)
pd.crosstab(results_df_all['ua_bin'], results_df_all['ru_bin'])

results_df_all['ur_mDeBERTa'] = results_df_all['pro-ukrainian-mDeBERTa'] - results_df_all['pro-russian-mDeBERTa']
results_df_all['ua_mDeBERTa'] = results_df_all['ur_mDeBERTa'].apply(lambda x: 1 if x > 0.15 else 0)
results_df_all['ru_mDeBERTa'] = results_df_all['ur_mDeBERTa'].apply(lambda x: 1 if x < -0.15 else 0)

In [ ]:

results_df_all['ur_custom'] = results_df_all['pro-ukrainian-custom'] - results_df_all['pro-russian-custom']
results_df_all['ua_custom'] = results_df_all['ur_custom'].apply(lambda x: 1 if x > 0.95 else 0)
results_df_all['ru_custom'] = results_df_all['ur_custom'].apply(lambda x: 1 if x < -0.95 else 0)

In [ ]:
pd.crosstab(results_df_all['ua_mDeBERTa'], results_df_all['ru_mDeBERTa'])

In [ ]:
pd.crosstab(results_df_all['ua_custom'], results_df_all['ru_custom'])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

columns = ['ua_mDeBERTa','ru_mDeBERTa']
corr_dict = {
    'ua_mDeBERTa':'ua_bin',
    'ru_mDeBERTa':'ru_bin'
}

for column in columns:
    conf_matrix = confusion_matrix(results_df_all[corr_dict[column]], results_df_all[column])
    accuracy = accuracy_score(results_df_all[corr_dict[column]], results_df_all[column])
    f1 = f1_score(results_df_all[corr_dict[column]], results_df_all[column], average='macro')

    print(f'Column: {column}')
    print(f'Accuracy: {accuracy}')
    print(f'F1 Score: {f1}')

    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['NO', 'YES'], yticklabels=['NO', 'YES'])
    plt.xlabel('DeBERTa')
    plt.ylabel('ChatGPT')
    plt.title(f'Confusion Matrix for {column}')
    plt.show()

In [ ]:
columns = ['ua_custom','ru_custom']
corr_dict = {
    'ua_custom':'ua_bin',
    'ru_custom':'ru_bin'
}

for column in columns:
    conf_matrix = confusion_matrix(results_df_all[corr_dict[column]], results_df_all[column])
    accuracy = accuracy_score(results_df_all[corr_dict[column]], results_df_all[column])
    f1 = f1_score(results_df_all[corr_dict[column]], results_df_all[column], average='macro')

    print(f'Column: {column}')
    print(f'Accuracy: {accuracy}')
    print(f'F1 Score: {f1}')

    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['NO', 'YES'], yticklabels=['NO', 'YES'])
    plt.xlabel('Custom')
    plt.ylabel('ChatGPT')
    plt.title(f'Confusion Matrix for {column}')
    plt.show()

## 3. Statistics





Hypotheses testing, e.g.:
- H1: Emotional content has an effect on forwards.
- H0: Emotional content has no effect on forwards.

Or:
- H1: Pro-Ukrainian/Pro-Russian content will get more forwards in deoccupied/occupied channels. (IE The interaction between content type and channel type has an effect on forwards.)

We want to control for how many people viewed the message, for instance, and other linguistic categories.

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(results_df_all['forwards'], kde=True, bins=10, color='skyblue')
plt.title('Distribution of Forwards')
plt.xlabel('Forwards')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print(f"Mean of forwards: {results_df_all['forwards'].mean()}")
print(f"Standard deviation of forwards: {results_df_all['forwards'].std()}")

In [ ]:
results_df_all['log_forwards'] = np.log(1 + results_df_all['forwards'])
plt.figure(figsize=(8, 6))
sns.histplot(results_df_all['log_forwards'], kde=True, bins=10, color='skyblue')
plt.title('Distribution of Log Forwards')
plt.xlabel('Log Forwards')
plt.ylabel('Frequency')
plt.show()

We could try using [negative binomial regression](https://en.wikipedia.org/wiki/Poisson_regression), which is good for overdispersed count data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
results_df_all['views_scaled'] = scaler.fit_transform(results_df_all[['views']])

results_df_all['occupied'] = results_df_all['category'].apply(lambda x: 1 if x == 'occupied' else 0)
results_df_all["pro-ru in occupied"] = results_df_all['pro-russian']*results_df_all['occupied']
results_df_all["pro-ua in occupied"] = results_df_all['pro-ukrainian']*results_df_all['occupied']

In [ ]:
# dependent variable
dv = 'forwards'

# independent variables
predictors = ['pro-ukrainian', 'pro-russian',
              'views_scaled', # control variable
              'discrediting', 'emotion', 'polarization',
              'impersonation', 'conspiracy', 'trolling']

In [ ]:
import statsmodels.api as sm

X = results_df_all[['occupied','pro-ru in occupied',"pro-ua in occupied"]+predictors]
X = sm.add_constant(X)           # Add constant for the intercept
y = results_df_all[dv]

model = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit()

print(model.summary())

In [ ]:
def dw_plot(model):
    err_series = model.params - model.conf_int()[0]
    coef_df = pd.DataFrame({'coef': np.exp(model.params.values[1:]),
                        'err': err_series.values[1:],
                        'varname': err_series.index.values[1:]
                       })
    coef_df = coef_df[coef_df['varname'] != 'views_scaled']
    fig, ax = plt.subplots(figsize=(8, 5))


    coef_df.plot(x='varname', y='coef', kind='barh',
             ax=ax, color='none',
             xerr='err', legend=False, linewidth=1.5)


    ax.scatter(coef_df['coef'], np.arange(coef_df.shape[0]),
           marker='s', s=120, color='black')


    ax.axvline(x=1, linestyle='--', color='black', linewidth=2)


    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.yaxis.set_ticks_position('none')

    plt.tight_layout()

    ax.grid(True, axis='x', linestyle='--', alpha=0.6)

    plt.show()

dw_plot(model)

For a unit increase in trolling, the expected forwards rate increases by around exp(0.6013) - 1 = 82.45%.

Occupied

In [ ]:
X = results_df_all[results_df_all['occupied']==1][predictors]
X = sm.add_constant(X)
y = results_df_all[results_df_all['occupied']==1][dv]

model = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit()

print(model.summary())

In [ ]:
dw_plot(model)

In [ ]:
X = results_df_all[results_df_all['occupied']==0][predictors]
X = sm.add_constant(X)
y = results_df_all[results_df_all['occupied']==0][dv]

model = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit()

print(model.summary())

In [ ]:
dw_plot(model)